In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 10), reused 23 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 6.89 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/kaggle/working/agnews-classifier


In [25]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=6,
    lora_alpha=18,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query','key','value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 925,444 || all params: 125,574,152 || trainable%: 0.7370


In [26]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [27]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='sgd',
    learning_rate=5e-3,
    lr_scheduler_type='cosine',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.388900,1.386846,0.226562
200,1.385400,1.380215,0.287500
300,1.383500,1.378506,0.418750
400,1.381500,1.384044,0.259375
500,1.382700,1.375127,0.278125
600,1.376500,1.381309,0.239063
700,1.372000,1.374159,0.425000
800,1.373100,1.372520,0.239063
900,1.369500,1.367138,0.306250
1000,1.363400,1.363329,0.429688


In [28]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.21it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [30]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3889,1.386846,0.254375,0.226562,0.004969,0.013405,0.002054,1.001481,-0.027813,0.890663
1,200,1.3854,1.380215,0.268304,0.287500,0.004878,0.026810,0.005185,1.003757,0.019196,1.071547
2,300,1.3835,1.378506,0.286161,0.418750,0.004728,0.040214,0.004994,1.003623,0.132589,1.463339
3,400,1.3815,1.384044,0.317411,0.259375,0.004523,0.053619,-0.002544,0.998162,-0.058036,0.817159
4,500,1.3827,1.375127,0.278125,0.278125,0.004268,0.067024,0.007573,1.005507,0.000000,1.000000
5,600,1.3765,1.381309,0.291518,0.239063,0.003969,0.080429,-0.004809,0.996519,-0.052455,0.820061
6,700,1.3720,1.374159,0.289732,0.425000,0.003635,0.093834,-0.002159,0.998429,0.135268,1.466872
7,800,1.3731,1.372520,0.351339,0.239063,0.003273,0.107239,0.000580,1.000422,-0.112277,0.680432
8,900,1.3695,1.367138,0.301339,0.306250,0.002891,0.120643,0.002362,1.001728,0.004911,1.016296
9,1000,1.3634,1.363329,0.376786,0.429688,0.002500,0.134048,0.000071,1.000052,0.052902,1.140403
